## functionalities

1. build a backing track, specify functional harmony, then generate improvisation.
* notes
* rhythm

2. given a backing track defined, do reharmonization.

3. given a chord and a corresponding chord name, figure out the closest chord in terms of voicing.

In [1]:
from midiutil import MIDIFile
import numpy as np
import pygame
import pandas as pd
from collections import deque 
from functools import reduce


pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


## what is a note:

note has properties of:
* name: A B C D 
* shift and n: actual numbers
* spectrum



## what is a chord/mode

it's a lot more complicated. Let's start with use cases.

When formulating a song, the input would always be using modes first. i.e.:

* We're in C Ionian key, the chord progression is IV, III, II, I.
* Sometimes we can add secondary dominance. F Ionian(II V)C Ionian(IV III II I).
* Sometimes we can even add more passing chords that points to no modes. It's just a brush of color.
* Even when the chords are in the mode, we can do alternation. For example V alt dominant chord.

So there are two ways to set up chord progressions:

1. functional harmony framework, with the freedom of altered notes
2. non-functional chords



1. modeling basic music components.
2. design input and output and add-ins. 
3. Variations, pattern recognition

* Input would be formulating a song. 
* Output would be midi files as well as a dataframe to represent the song and solos.
* add-ins: customized patterns for backing tracks.
* Pattern

voicing module is for customized chord voicings and


A solo module.

In [2]:
# a systematic representation of the known knowledge makes me appreciate more about the "stars" for real music. 

# Understand the limitations to push the boundaries.

In [3]:
class Note:
    """
    name is the note of a chord.
    n represent relative location in the twelve equal temperament
    shift = 0 means 4th octave.
    """
    def __init__(self,name=None,n=None,shift=0,*args, **kwargs):
        note_name_l = ['C','C#','Db','D','D#',\
                              'Eb','E','F','F#','Gb','G','G#','Ab','A','A#','Bb','B']
        note_loc_l = [0, 1, 1, 2, 3, 3, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11]
        convert_dict = dict(zip(note_name_l,note_loc_l))
        rev_convert_dict = dict(zip(note_loc_l,note_name_l))
        if name and (n is None):
            self.name = name
            self.shift = shift
            self.n = convert_dict[self.name]+(self.shift+5)*12
        elif (n is not None) and (name is None):
            self.n = n%12
            if shift is None:
                self.shift = n//12-5
            else:
                self.shift = shift
            self.name = rev_convert_dict[self.n]
            
        self.repr_str = f"Note: {self.name}"
        self.spectrum = [self.n+k for k in [12*(i-3) for i in range(7)]]
            
    def __repr__(self):
        return self.repr_str 
    
    def __str__(self):
        return self.repr_str 
    
    def __sub__(self,other_note):
        """
        measure the distance of two notes. Important right?
        
        other_note is always the chord notes (or extensions); self note is the root.
        """
        
        return (other_note.n%12-self.n)%12
    
    def __members(self):
        return (self.name)
    
    def __eq__(self, other):
        if type(other) is type(self):
            return self.__members() == other.__members()
        else:
            return False

    def __hash__(self):
        return hash(self.__members())


In [4]:
print(Note("C").n, Note(n=71),Note("D")-Note("C"))

60 Note: B 10


In [107]:
class Chord:
    """
    two ways to think about a chord.
    
    1. a chord is given by quality. thus in terms of modes, it's the 1st degree of the mode that corresponds to that quality
    2. a chord is a functional chord, given a mode.
    
    extensions: list of extensions: ['9','#11']
    """
    
    QUALITY_MOD_D = dict(zip(['Maj7','m7','7','m7b5'],['Ionian','Dorian','Mixolydian','Locrian']))
    
    MODE_LIST = ['Ionian','Dorian','Phrygian','Lydian','Mixolydian','Aeolian','Locrian']
    MODE_SEC = range(7)

    MOD_DEGREE_D = dict(zip(MODE_LIST,MODE_SEC))
    DEGREE_MOD_D = dict(zip(MODE_SEC,MODE_LIST))

    DIATONIC_CHORD = ['Maj7','m7','m7','Maj7','7','m7','m7b5']

    CIRCLE_OF_FIFTH = [2,2,1,2,2,2,1]

    QUALITY_MODE_LIST = ['Ionian','Dorian','Mixolydian','Locrian']
    QM_DISTANCE = [frozenset([4,7,11]),frozenset([3,7,10]),frozenset([4,7,10]),frozenset([3,6,10])]
    
    
    ROMAN_N_DICT = dict(zip(range(1,8),["I","II","III","IV","V","VI","VII"]))
    
    def __init__(self,root, quality = None, key_root = None, key_mode = None, mode_degree = None, extensions = None):
        
        self.root = root
        self.quality = quality
        self.key_mode = key_mode
        self.mode_degree = mode_degree
        self.extensions = extensions
        self.roman = None

        self.mode = None
        
        if self.quality in self.QUALITY_MOD_D.keys():
            self.mode = self.QUALITY_MOD_D[self.quality]
        
        if key_mode is not None:
            self.roman = self.ROMAN_N_DICT[mode_degree]
            
        if self.key_mode:  
            self.repr_str = f"{self.root}{self.quality} - ({self.roman}{self.quality})"
        else:
            self.repr_str = f"{self.root}{self.quality}"
            

    @classmethod
    def from_quality(cls,root, quality,extensions=None):
        
        return cls(root, quality, extensions)
        
    @classmethod
    def from_function(cls, key_root, key_mode, mode_degree, extensions=None):
        ## get quality
        shift_n = cls.MOD_DEGREE_D[key_mode]+(mode_degree-1)
        tmp_diatonic_chord = deque(cls.DIATONIC_CHORD) 
        tmp_diatonic_chord.rotate(-1*shift_n)

        tmp_quality = tmp_diatonic_chord[0]

        ##. figure out the root 
        tmp_mode_sec = cls.MOD_DEGREE_D[key_mode]
        mode_distance_deque = deque(cls.CIRCLE_OF_FIFTH)
        mode_distance_deque.rotate(-1*(tmp_mode_sec))

        distance_list = [0]+list(mode_distance_deque)
        realized_distance = sum(distance_list[:mode_degree])
        key_note_n = Note(key_root).n+realized_distance

        root = Note(n = key_note_n).name
        return cls(root, tmp_quality, extensions,key_mode, mode_degree,extensions)
        
            
    def __repr__(self):
        if self.extensions:
            return self.repr_str +' ext: ['+'-'.join([str(i) for i in self.extensions])+"]"
        else:
            return self.repr_str  
    
    def __str__(self):
        if self.extensions:
            return self.repr_str +' ext: ['+'-'.join([str(i) for i in self.extensions])+"]"
        else:
            return self.repr_str 
        
    def get_notes(self):
        """
        return a list of notes that can be played over this chord.
        """
        if self.mode_degree is None:
            # if a chord doesn't have functionality, then solo over the scales that fit that quality.
            raise NotImplementedError
            
        else:
            # first get scales based on quality, then look at base mode. return the overlaping notes.
            tmp_rotate = self.MOD_DEGREE_D[self.QUALITY_MOD_D[self.quality]]
            mode_distance_deque = deque(self.CIRCLE_OF_FIFTH)
            mode_distance_deque.rotate(-1*(tmp_rotate))

            distance_list = [0]+list(mode_distance_deque)
            distance_list = np.cumsum(distance_list)

            num_root = Note(k.root).n
            n_root = Note(n = num_root)
            n_3rd = Note(n = num_root+distance_list[2])
            n_5th = Note(n = num_root+distance_list[4])
            n_7th = Note(n = num_root+distance_list[6])
            n_9th = Note(n = num_root+distance_list[1])
            notes_dict = {"n_root":n_root,"n_3rd":n_3rd,"n_5th":n_5th,"n_7th":n_7th,"n_9th":n_9th}
            return notes_dict

    
    

In [108]:
k = Chord.from_quality('F','Maj7')
k,k.root,k.quality, k.key_mode,k.roman

(FMaj7, 'F', 'Maj7', None, None)

In [109]:
k = Chord.from_function(key_root = 'A',key_mode = 'Aeolian',mode_degree = 1)
k,k.root,k.quality, k.key_mode,k.roman

(Am7 - (Im7), 'A', 'm7', 'Aeolian', 'I')

In [110]:
k.get_notes()

{'n_root': Note: A,
 'n_3rd': Note: C,
 'n_5th': Note: E,
 'n_7th': Note: G,
 'n_9th': Note: B}

In [111]:
input_df = pd.DataFrame({"bar":[1,2,3,4],"key":['C']*4,"mode":['Ionian']*4,"degree":[2,5,1,1],"bar_time":[1]*4,'extensions':[None,[13],None,None]})

In [112]:
class Song:
    def __init__(self):
        self.input_df = None
        self.output_df = None
        
    def analyze(self,input_df):
        self.input_df = input_df.copy()        
        input_df['chord'] = input_df.apply(lambda x: (Chord.from_function(key_root = x.key, key_mode = x['mode'], mode_degree = x.degree,extensions = x.extensions)), axis = 1)
        self.output_df = input_df.copy()
    

In [113]:
my_song = Song()



In [114]:
my_song.analyze(input_df)

output_df = my_song.output_df

In [115]:
output_df 

,bar,key,mode,degree,bar_time,extensions,chord
0,1,C,Ionian,2,1,None,Dm7 - (IIm7)
1,2,C,Ionian,5,1,[13],G7 - (V7) ext: [13]
2,3,C,Ionian,1,1,None,CMaj7 - (IMaj7)
3,4,C,Ionian,1,1,None,CMaj7 - (IMaj7)


In [ ]:
my_song.improvise()
# step 1: from note names to numeric characteristic notes
# step 2: use characteristic notes to line up a melody structure. 
# step 3: look at key mode, add additional notes that target those structure notes

# what about rhythm????